# Pandas :  Mezcla Datasets: Merge /  Join

Una característica esencial que ofrece Pandas son sus operaciones de **Join** y **Merge** en memoria de alto rendimiento. Si alguna vez ha trabajado con bases de datos, debe estar familiarizado con este tipo de interacción de datos. La interfaz principal para esto es la función **pd.merge**, y veremos algunos ejemplos de cómo esto puede funcionar en la práctica.

In [2]:
import pandas as pd
import numpy as np

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Relational Algebra

Pandas implementa varios de estos bloques de construcción fundamentales en la función **pd.merge()** y el método **join()** relacionado de Series y Dataframes. Como veremos, estos le permiten vincular de manera eficiente datos de diferentes fuentes

# Joins

La función **pd.merge()** implementa varios tipos de uniones: uniones de uno a uno, de muchos a uno y de muchos a muchos. Se accede a los tres tipos de uniones a través de una llamada idéntica a la interfaz **pd.merge();** el tipo de unión realizada depende de la forma de los datos de entrada. 
Aquí mostraremos ejemplos simples de los tres tipos de fusiones y discutiremos las opciones detalladas más adelante.

## Joins uno a uno / inner Join

Quizás el tipo más simple de expresión de combinación es la combinación uno a uno, que en muchos aspectos es muy similar a la concatenación por columnas que se ve en Combinación de conjuntos de datos: **Concat & Append**. Como ejemplo concreto, considere los siguientes dos DataFrames que contienen información sobre varios empleados en una empresa.

In [4]:
df1 = pd.DataFrame({'employee': ['Sara', 'Juan', 'Pedro', 'Ana'],
                    'group': ['Ingeniero', 'Veterninario', 'Ingeniero', 'Odontologo']})
df2 = pd.DataFrame({'employee': ['Ana', 'Pedro', 'Sara', 'Juan'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Sara,Ingeniero
1,Juan,Veterninario
2,Pedro,Ingeniero
3,Ana,Odontologo
,employee,hire_date
0,Ana,2004
1,Pedro,2008
2,Sara,2012
3,Juan,2014


Para combinar esta información en un solo DataFrame, podemos usar la función **pd.merge()**:

In [5]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Sara,Ingeniero,2012
1,Juan,Veterninario,2014
2,Pedro,Ingeniero,2008
3,Ana,Odontologo,2004


## Joins Muchos a uno / many-to-one

Las uniones de muchos a uno son uniones en las que una de las dos columnas clave contiene entradas duplicadas. Para el caso de muchos a uno, el DataFrame resultante conservará esas entradas duplicadas según corresponda. Considere el siguiente ejemplo de una combinación de muchos a uno:

In [6]:
df4 = pd.DataFrame({'group': ['Ingeniero', 'Odontologo', 'Veterninario'],
                    'supervisor': ['Veronica', 'Rodrigo', 'Cesar']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee         group  hire_date
0     Sara     Ingeniero       2012
1     Juan  Veterninario       2014
2    Pedro     Ingeniero       2008
3      Ana    Odontologo       2004

df4
          group supervisor
0     Ingeniero   Veronica
1    Odontologo    Rodrigo
2  Veterninario      Cesar

pd.merge(df3, df4)
  employee         group  hire_date supervisor
0     Sara     Ingeniero       2012   Veronica
1    Pedro     Ingeniero       2008   Veronica
2     Juan  Veterninario       2014      Cesar
3      Ana    Odontologo       2004    Rodrigo

El DataFrame resultante tiene una columna adicional con la información del ``supervisor``, donde la información se repite en una o más ubicaciones según lo requieran las entradas.

In [7]:
df5 = pd.DataFrame({'group': ['Ingeniero', 'Ingeniero',
                              'Odontologo', 'Odontologo', 'Veterninario', 'Veterninario'],
                    'skills': ['Matematicas', 'Programacion', 'Medicina', 'Anotomia',
                               'Animales', 'Salud']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Sara,Ingeniero
1,Juan,Veterninario
2,Pedro,Ingeniero
3,Ana,Odontologo
,group,skills
0,Ingeniero,Matematicas
1,Ingeniero,Programacion
2,Odontologo,Medicina
3,Odontologo,Anotomia
4,Veterninario,Animales


Estos tres tipos de uniones se pueden usar con otras herramientas de Pandas para implementar una amplia gama de funciones. Pero en la práctica, los conjuntos de datos rara vez son tan limpios como con el que estamos trabajando aquí. 
En la siguiente sección consideraremos algunas de las opciones proporcionadas por pd.merge() que le permiten ajustar cómo funcionan las operaciones de combinación.

## Especificación de la clave de combinación

Ya hemos visto el comportamiento predeterminado de ``pd.merge()``: busca uno o más nombres de columna coincidentes entre las dos entradas y lo usa como clave. 

Sin embargo, a menudo los nombres de las columnas no coincidirán tan bien y ``pd.merge()`` proporciona una variedad de opciones para manejar esto.

In [8]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee         group
0     Sara     Ingeniero
1     Juan  Veterninario
2    Pedro     Ingeniero
3      Ana    Odontologo

df2
  employee  hire_date
0      Ana       2004
1    Pedro       2008
2     Sara       2012
3     Juan       2014

pd.merge(df1, df2, on='employee')
  employee         group  hire_date
0     Sara     Ingeniero       2012
1     Juan  Veterninario       2014
2    Pedro     Ingeniero       2008
3      Ana    Odontologo       2004

Esta opción solo funciona si tanto el ``DataFrame`` izquierdo como el derecho tienen el nombre de columna especificado.

## Las palabras clave left_on y right_on

En ocasiones, es posible que desee fusionar dos conjuntos de datos con diferentes nombres de columna; por ejemplo, podemos tener un conjunto de datos en el que el nombre del empleado esté etiquetado como "nombre" en lugar de "empleado". En este caso, podemos usar las palabras clave left_on y right_on para especificar los dos nombres de columna:

In [9]:
df3 = pd.DataFrame({'name': ['Pedro', 'Sara', 'Ana', 'Juan'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee         group
0     Sara     Ingeniero
1     Juan  Veterninario
2    Pedro     Ingeniero
3      Ana    Odontologo

df3
    name  salary
0  Pedro   70000
1   Sara   80000
2    Ana  120000
3   Juan   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee         group   name  salary
0     Sara     Ingeniero   Sara   80000
1     Juan  Veterninario   Juan   90000
2    Pedro     Ingeniero  Pedro   70000
3      Ana    Odontologo    Ana  120000

El resultado tiene una columna redundante que podemos quitar si lo deseamos, por ejemplo, usando el método ``drop()`` de DataFrames:

In [10]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Sara,Ingeniero,80000
1,Juan,Veterninario,90000
2,Pedro,Ingeniero,70000
3,Ana,Odontologo,120000


## Las palabras clave left_index y right_index

A veces, en lugar de fusionarse en una columna, le gustaría fusionarse en un índice. Por ejemplo, sus datos podrían verse así:

In [11]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Sara,Ingeniero
Juan,Veterninario
Pedro,Ingeniero
Ana,Odontologo
,hire_date
employee,
Ana,2004
Pedro,2008
Sara,2012


Puede usar el índice como la clave para fusionar especificando las etiquetas de parametros ``left_index`` y/o ``right_index`` en ``pd.merge()``:

In [12]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Sara,Ingeniero
Juan,Veterninario
Pedro,Ingeniero
Ana,Odontologo
,hire_date
employee,
Ana,2004
Pedro,2008
Sara,2012


Para mayor comodidad, ``DataFrame``s implementa el método ``join()``, que realiza una combinación que por defecto se une a los índices:

In [13]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Sara,Ingeniero
Juan,Veterninario
Pedro,Ingeniero
Ana,Odontologo
,hire_date
employee,
Ana,2004
Pedro,2008
Sara,2012


Si desea mezclar índices y columnas, puede combinar ``left_index`` con ``right_on`` o ``left_on`` con ``right_index`` para obtener el comportamiento deseado:

In [14]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                 group
employee              
Sara         Ingeniero
Juan      Veterninario
Pedro        Ingeniero
Ana         Odontologo

df3
    name  salary
0  Pedro   70000
1   Sara   80000
2    Ana  120000
3   Juan   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
          group   name  salary
1     Ingeniero   Sara   80000
3  Veterninario   Juan   90000
0     Ingeniero  Pedro   70000
2    Odontologo    Ana  120000

## Example: US States Data

Las operaciones de Merge y Join surgen con mayor frecuencia cuando se combinan datos de diferentes fuentes.
Aquí consideraremos un ejemplo de algunos datos sobre los estados de EE. UU. y sus poblaciones.
Los archivos de datos se pueden encontrar en http://github.com/jakevdp/data-USstates/:

In [17]:
pop = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv')
areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv')
abbrevs = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Dada esta información, supongamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EE. UU. por su densidad de población de 2010.

Claramente tenemos los datos aquí para encontrar este resultado, pero tendremos que combinar los conjuntos de datos para encontrar el resultado.

Comenzaremos con un Merge de muchos a uno que nos dará el nombre completo del estado dentro de la población ``DataFrame``.
Queremos fusionar según la columna ``estado/región`` de ``pop`` y la columna ``abreviatura`` de ``abbrevs``.
Usaremos ``how='outer'`` para asegurarnos de que no se desechen datos debido a etiquetas que no coinciden.

In [18]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Verifiquemos dos veces si hubo discrepancias aquí, lo que podemos hacer buscando filas con valores nulos:

In [20]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Parte de la información de ``población`` es nula; Para averiguar esto, veamos esta función.

In [21]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Parece que todos los valores de población nula son de Puerto Rico antes del año 2000; esto probablemente se deba a que estos datos no están disponibles en la fuente original.

Más importante aún, vemos también que algunas de las nuevas entradas ``state`` también son nulas, lo que significa que no había ninguna entrada correspondiente en la clave ``abbrevs``.
Averigüemos qué regiones carecen de esta coincidencia:

In [22]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Podemos inferir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (EE. UU.), mientras que estas entradas no aparecen en la clave de abreviatura de estado. Podemos solucionarlos rápidamente completando las entradas correspondientes:

In [23]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No más valores nulos en la columna ``estado``: ¡ya estamos listos!

Ahora podemos hacer un Merge del resultado con los datos del área usando un procedimiento similar.
Examinando nuestros resultados, desearemos unirnos en la columna ``estado`` en ambos:

In [24]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Nuevamente, verifiquemos si hay valores nulos para ver si hubo discrepancias:

In [25]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [26]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Vemos que nuestro ``áreas`` ``DataFrame`` no contiene el área de los Estados Unidos como un todo.
Podríamos insertar el valor apropiado (usando la suma de todas las áreas del estado, por ejemplo), pero en este caso simplemente eliminaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra discusión actual:

In [ ]:
final.dropna(inplace=True)
final.head()

Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta de interés, seleccionemos primero la porción de datos correspondiente al año 2000 y la población total.
Usaremos la función ``query()`` para hacer esto rápidamente (esto requiere que se instale el paquete ``numexpr``; consulte [Pandas de alto rendimiento: ``eval()`` y ``query( )``](03.12-Performance-Eval-and-Query.ipynb)):

In [30]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Calculemos la densidad de población y mostrémosla en orden.
Comenzaremos por volver a indexar nuestros datos en el estado y luego calcularemos el resultado:

In [31]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [32]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

El resultado es una clasificación de los estados de EE. UU. más Washington, DC y Puerto Rico en orden de densidad de población en 2010, en residentes por milla cuadrada.
Podemos ver que, con mucho, la región más densa en este conjunto de datos es Washington, DC (es decir, el Distrito de Columbia); entre los estados, el más denso es Nueva Jersey.

También podemos consultar el final de la lista:

In [33]:
density.tail()

state
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
United States         NaN
dtype: float64

Vemos que el estado menos denso, por mucho, es Alaska, con un promedio de poco más de un residente por milla cuadrada.

Este tipo de combinación de datos desordenada es una tarea común cuando se intenta responder preguntas utilizando fuentes de datos del mundo real.